### Clustering using PseAAC features

##### <u> Part 1: Extracting protein sequence data </u>

In [29]:
import re

class CAZy_data:
    def __init__(self,filename1,filename2):
        self.data,self.acc,self.seq=[],[],[]
        with open(filename1,'r',encoding='utf-8') as inpt:
            for each in inpt:
                self.data.append(each.rstrip().split('$'))
        with open(filename2,'r',encoding='utf-8') as inpt1:
            for each1 in inpt1:
                if each1.startswith('>'):
                    self.acc.append(each1.rstrip())
                else:
                    self.seq.append(each1.rstrip())

    def export_to_txt(self, output_filename):
        with open(output_filename, 'w', encoding='utf-8') as output_file:
            for line in self.fasta:
                output_file.write(line + '\n')
            
                  
    def data_fetch(self,typ,position):
        typ_data=[]
        if typ=='all':
            typ_data=self.data
        else:
            for each in self.data:
                mult=each[position].split(' ')
                if len(mult)==1:#### In case typ = EC, Multi EC number and protein with no EC number are ignore.
                    if mult[0]==typ:
                        typ_data.append(each)
        return typ_data
    
    def EC_GH(self,ec_no,gh_fam):
        self.fasta=[]
        cazy_ec=self.data_fetch(ec_no,1)
        cazy_gh=self.data_fetch(gh_fam,-1)
        self.common_data=[[i[0],i[1],i[3],i[4],i[-2],i[-1]] for i in cazy_ec if i in cazy_gh]
        rm_prt, rm_prt_fasta=[],[]
        for each in range(len(self.common_data)):
            t=self.common_data[each]
            if self.prtn_filter(t[0]):
                all_acc=t[3].split(' ')
                if all_acc[0]!='':
                    for e_acc in all_acc:
                        e_seq=self.seq_fetch(e_acc)
                        try:
                            create_error=0/len(e_seq) # to remove accession number which doesnt have hits
                            self.fasta.append(f'>{e_acc}${t[0]}${t[1]}${t[2]}${t[-2]}${t[-1]}')
                            self.fasta.append(e_seq[0])
                        except ZeroDivisionError:
                            rm_prt_fasta.append(e_acc)
            else:
                rm_prt.append(t)
        print('Total number of sequences:',len(self.fasta)/2)
        print('Number of removed partial or fragment proteins (CAZy):',len(rm_prt))
        print('Number of removed partial or fragment proteins (Fasta):',len(rm_prt_fasta))
        return self.fasta
    def prtn_filter(self,prt_name):
        hit=1
        if re.search('partial|fragment',prt_name.lower()):
            hit-=1
        return hit
            
    def seq_fetch(self,accession):
        hits=[]
        temp=0
        for each in range(len(self.acc)):
            if re.search(f'{accession}\D',self.acc[each]):
                temp+=1
                
                if self.prtn_filter(self.acc[each]): # remove partial| fragment accession numbers from GenBank description
                    hits.append(self.seq[each])
        if temp>1:
            print(f'Multiple hits for {accession}')
        elif temp==0:
            print(f'No hits for {accession}')
        return hits

In [30]:
import os

output_folder = 'outputs'
os.makedirs(output_folder, exist_ok=True)

GH_family_list = [5, 5, 5, 5, 10, 26, 51, 148, 1, 1, 2, 30, 39, 39, 3, 3, 6, 6, 44, 13, 7, 7, 16, 12, 131, 8, 48, 48, 48, 116, 9, 9, 9, 45, 124]
EC_numbers_list = ['3.2.1.4', '3.2.1.21', '3.2.1.74', '3.2.1.91', '3.2.1.4', '3.2.1.4', '3.2.1.4', '3.2.1.4', '3.2.1.21', '3.2.1.74', '3.2.1.21', '3.2.1.21', '3.2.1.21', '3.2.1.74', '3.2.1.21', '3.2.1.74', '3.2.1.4', '3.2.1.91', '3.2.1.4', '3.2.1.21', '3.2.1.4', '3.2.1.176', '3.2.1.21', '3.2.1.4', '3.2.1.21', '3.2.1.4', '3.2.1.4', '3.2.1.91', '3.2.1.176', '3.2.1.21', '3.2.1.4', '3.2.1.74', '3.2.1.91', '3.2.1.4', '3.2.1.4']

In_data = CAZy_data('cazy_char_10_6_22.txt', 'char_gh_23_6_22.txt')

for GH_family, EC_number in zip(GH_family_list, EC_numbers_list):
    cazy_acc_seq = In_data.EC_GH(EC_number, f'GH{GH_family}')
    output_filename = os.path.join(output_folder, f'output_GH{GH_family}_EC{EC_number.replace(".", "_")}.txt')
    In_data.export_to_txt(output_filename)

print(f'Individual output files are stored in the {output_folder} folder.')


No hits for NP_126623.1
No hits for NP_143072.1
No hits for AAU23613.1
No hits for NP_389695.1
No hits for NP_622045.1
No hits for CAB01405.1
No hits for ABZ70413.1
No hits for AAD48494.2
No hits for NP_241469.1
No hits for WP_018063499.1
No hits for AAC02964.1
No hits for BAA12826.1
No hits for AAZ56745.1
No hits for AAZ54939.1
No hits for AHA42547.1
No hits for NP_638867.1
No hits for NP_298108.1
No hits for XP_324942.1
No hits for XP_002475436.1
No hits for AAL33630.1
No hits for AAL33639.1
Total number of sequences: 381.0
Number of removed partial or fragment proteins (CAZy): 15
Number of removed partial or fragment proteins (Fasta): 50
No hits for NP_012272.1
Total number of sequences: 3.0
Number of removed partial or fragment proteins (CAZy): 0
Number of removed partial or fragment proteins (Fasta): 3
Total number of sequences: 8.0
Number of removed partial or fragment proteins (CAZy): 0
Number of removed partial or fragment proteins (Fasta): 0
Total number of sequences: 0.0
Numb

In [32]:
import os

# Create an 'outputs' folder if it doesn't exist
output_folder = 'outputs'
os.makedirs(output_folder, exist_ok=True)

# List all files in the 'outputs' folder
file_list = [file for file in os.listdir(output_folder) if file.endswith('.txt')]

# Combine files into a single 'output.txt' file
combined_output_path = os.path.join(output_folder, 'output.txt')
with open(combined_output_path, 'w', encoding='utf-8') as combined_output_file:
    for file in file_list:
        file_path = os.path.join(output_folder, file)
        with open(file_path, 'r', encoding='utf-8') as input_file:
            content = input_file.read()
            combined_output_file.write(content)
            # Add a newline for better separation of file content
            combined_output_file.write('\n')

print(f'Combined output file is stored at: {combined_output_path}')


Combined output file is stored at: outputs\output.txt
